## La MUDANZA: an application of the Google Maps API

In [11]:
# Standard libraries
from datetime import datetime
import itertools
import json
import re
import time
# Third-party libraries
import googlemaps
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import folium
from folium.features import DivIcon

# Typing for type hinting
from typing import Union, List, Dict, Any, Tuple

In [ ]:
credentials_path = ""
with open(credentials_path, 'r') as file:
    credentials = json.load(file)

In [4]:
gmaps = googlemaps.Client(key=credentials["api_key_maps"])

## Coordinates and reference poins

In [5]:
def coor_from_place(str_place: str) -> tuple[float, float]:
    """
    Retrieves the geographic coordinates (latitude and longitude) of a specified place in Colombia.

    Args:
        str_place (str): The name or description of the place (e.g., "Bogotá", "Calle 80, Medellín").

    Returns:
        tuple[float, float]: A tuple containing the latitude and longitude of the place in the format (lat, lng).
    """
    result = gmaps.geocode(str_place, components={"country": "CO"})
    dict_coor = result[0]['geometry']['location']
    return (dict_coor['lat'], dict_coor['lng'])

def get_time_from_coor(
    origen: tuple[float, float],
    destino: tuple[float, float],
    mode: str = 'transit',
    transit_mode: str = 'train'
) -> Union[str, int]:
    """
    Retrieves the estimated travel time between two geographic coordinates using the Google Maps API.

    Args:
        origen (tuple[float, float]): Origin coordinates as (latitude, longitude).
        destino (tuple[float, float]): Destination coordinates as (latitude, longitude).
        mode (str, optional): Mode of transportation (e.g., 'driving', 'walking', 'transit'). Defaults to 'transit'.
        transit_mode (str, optional): Type of public transit (e.g., 'bus', 'train', 'subway'). Defaults to 'train'.

    Returns:
        Union[str, int]: 
            - If successful: Estimated travel time as a string (e.g., '1 hour 5 mins').
            - If the element status is not OK: Returns 0.
            - If the API response status is not OK: Returns -1.
    """
    if mode != "walking":
        respuesta = gmaps.distance_matrix(
            origins=[origen],
            destinations=[destino],
            mode=mode,
            transit_mode=transit_mode,
            departure_time=datetime.now()
        )
    else:
        respuesta = gmaps.distance_matrix(
            origins=[origen],
            destinations=[destino],
            mode=mode,
            departure_time=datetime.now()
        )

    if respuesta['status'] == 'OK':
        elemento = respuesta['rows'][0]['elements'][0]
        if elemento['status'] == 'OK':
            tiempo = elemento['duration']['text']
            return tiempo
        else:
            return 0
    else:
        return -1
    

def get_stations(radius: int = 15000) -> List[Dict[str, float]]:
    """
    Retrieves a list of nearby metro stations within a specified radius of Medellín, Colombia, 
    using the Google Places API.

    Args:
        radius (int, optional): Search radius in meters. Defaults to 15,000.

    Returns:
        List[Dict[str, float]]: A list of dictionaries, each containing:
            - 'name' (str): Station name
            - 'latitude' (float): Latitude
            - 'longitude' (float): Longitude
    """
    location = "6.2442,-75.5812"  # Coordinates for Medellín, Colombia
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params = {
        "location": location,
        "radius": radius,
        "keyword": "estación de metro",
        "key": credentials["api_key_maps"]
    }

    stations = []

    while True:
        response = requests.get(url, params=params)
        data = response.json()

        for result in data.get("results", []):
            stations.append({
                "name": result["name"],
                "latitude": result["geometry"]["location"]["lat"],
                "longitude": result["geometry"]["location"]["lng"]
            })

        if "next_page_token" in data:
            time.sleep(2)  # Required delay for Google Places API pagination
            params["pagetoken"] = data["next_page_token"]
        else:
            break

    return stations





## Cost function calculation

In [ ]:
def cost_function(
    time_station: int, 
    weight_station: float, 
    time_gym: int, 
    weight_gym: float, 
    time_work: int, 
    weight_work: float
) -> int:
    """
    Computes a weighted cost based on travel times to station, gym, and workplace.

    Args:
        time_station (int): Time to the nearest station in minutes.
        weight_station (float): Weight assigned to station time.
        time_gym (int): Time to the nearest gym in minutes.
        weight_gym (float): Weight assigned to gym time.
        time_work (int): Time to the workplace in minutes.
        weight_work (float): Weight assigned to work time.

    Returns:
        int: Total weighted cost rounded to the nearest integer.
    """
    cost_time = time_station * weight_station + time_gym * weight_gym + time_work * weight_work
    return int(cost_time)

def get_cost_place(
    apt_data: Dict[str, Any],
    gym_results: Dict[str, Any],
    station_results: list[Dict[str, Any]],
    index: int,
    weight_station: float = 0.2,
    weight_gym: float = 0.8
) -> Dict[tuple, Dict[str, Any]]:
    """
    Computes and ranks combinations of nearby gym and station locations for an apartment
    based on a weighted travel time cost function.

    Args:
        apt_data (dict): Dictionary containing travel times to stations, gyms, and work.
        gym_results (dict): Dictionary with gym location details from the Maps API.
        station_results (list): List of station dictionaries with names and coordinates.
        index (int): Index representing the current apartment being evaluated.
        weight_station (float): Weight for station travel time in the cost function.
        weight_gym (float): Weight for gym travel time in the cost function.

    Returns:
        dict: Sorted dictionary of combinations with keys as (station, gym) tuples and
              values containing time costs and location details.
    """
    combinations = itertools.product(
        apt_data['stations'][str(index)].items(),
        apt_data['gym'][str(index)].items()
    )

    result_dict = {
        (station_name, gym_name): {
            'time_cost': cost_function(
                station_time, weight_station,
                gym_time, weight_gym,
                apt_data['work'][station_name], weight_station
            ),
            'time_station': station_time,
            'time_gym': gym_time,
            'time_work': apt_data['work'][station_name],
            'gym_location': gym_results[gym_name][0]["geometry"]["location"],
            'station_location': {
                k: v for k, v in next(
                    item.items() for item in station_results if item['name'] == station_name
                ) if k in ('latitude', 'longitude')
            }
        }
        for (station_name, station_time), (gym_name, gym_time) in combinations
    }

    return dict(sorted(result_dict.items(), key=lambda item: item[1]['time_cost']))

#  Formatting functions

In [ ]:
def str_to_minutes(duration_str: str) -> int:
    """
    Converts a duration string (e.g., '1 day 2 hours 30 mins') into the total number of minutes.

    Args:
        duration_str (str): A string representing a time duration, typically in formats like 
                            '2 hours 15 mins', '1 day 3 hours', etc.

    Returns:
        int: Total duration expressed in minutes.
    """
    duration_str = duration_str.lower()

    days = int(re.search(r'(\d+)\s*day', duration_str).group(1)) if 'day' in duration_str else 0
    hours = int(re.search(r'(\d+)\s*hour', duration_str).group(1)) if 'hour' in duration_str else 0
    minutes = int(re.search(r'(\d+)\s*min', duration_str).group(1)) if 'min' in duration_str else 0

    total_minutes = days * 24 * 60 + hours * 60 + minutes
    return total_minutes

def sorted_time_cost(
    results_dict: Dict[Any, Dict[Any, Dict[str, Any]]],
    key: str = 'time_cost'
) -> Dict[Any, Dict[Any, Dict[str, Any]]]:
    """
    Sorts a nested dictionary by the minimum value of a given key (e.g., 'time_cost') 
    in the inner dictionaries.

    Args:
        results_dict (dict): A dictionary where values are nested dictionaries that 
                             contain cost-related metrics.
        key (str, optional): The key to sort by inside the nested dictionaries. 
                             Defaults to 'time_cost'.

    Returns:
        dict: The input dictionary sorted by the minimum value of the specified key 
              in each inner dictionary.
    """
    sorted_items = sorted(
        results_dict.items(),
        key=lambda item: min(subvalue[key] for subvalue in item[1].values())
    )

    return {outer_key: outer_value for outer_key, outer_value in sorted_items}

def get_top_n_keys(
    data: Dict[Any, Dict[Any, Any]],
    n: int = 3,
    from_end: bool = False
) -> Dict[Any, Dict[Any, Any]]:
    """
    Extracts the first or last 'n' keys from each sub-dictionary within a parent dictionary.

    Args:
        data (dict): A dictionary where each value is a sub-dictionary.
        n (int, optional): Number of keys to extract from each sub-dictionary. Defaults to 3.
        from_end (bool, optional): If True, selects the last 'n' keys; otherwise, selects the first 'n'. Defaults to False.

    Returns:
        dict: A new dictionary with the same outer keys but only the selected 'n' inner keys in each sub-dictionary.
    """
    filtered_data = {}

    for outer_key, inner_dict in data.items():
        inner_keys = list(inner_dict.keys())
        selected_keys = inner_keys[-n:] if from_end else inner_keys[:n]
        filtered_data[outer_key] = {k: inner_dict[k] for k in selected_keys}

    return filtered_data

## Map generation

In [91]:
def get_polyline(directions_result: List[Dict[str, Any]]) -> List[Tuple[float, float]]:
    """
    Extracts a sequence of (latitude, longitude) tuples representing the end locations
    of each step in a directions result.

    Args:
        directions_result (list): A list of direction results from the Google Maps Directions API.

    Returns:
        list: A list of (lat, lng) tuples representing the path of the route.
    """
    steps = directions_result[0]['legs'][0]['steps']
    polyline = [
        (step['end_location']['lat'], step['end_location']['lng'])
        for step in steps
    ]
    return polyline

def generate_map(
    result_dict: Dict[str, Any],
    index: int,
    house_location: Tuple[float, float],
    work_location: Tuple[float, float],
    map_name: str,
    line_color: str = 'black'
) -> None:
    """
    Generates an interactive map showing routes between a house, gym, station, and work location,
    with markers and polyline for each step of the journey.

    Args:
        result_dict (dict): Dictionary containing results with station, gym, and work locations.
        index (int): Index for the specific result to plot.
        house_location (tuple): Coordinates (lat, lng) of the house.
        work_location (tuple): Coordinates (lat, lng) of the work location.
        map_name (str): Name to save the resulting HTML map.
        line_color (str, optional): Color for the polyline routes. Defaults to 'black'.
    """
    # Extract the best result for the given index
    key_location = list(result_dict[str(index)].keys())[0]
    best_result = result_dict[str(index)][key_location]

    # Create the base map centered on the house
    map = folium.Map(location=house_location, zoom_start=14)

    # Get coordinates for the station and gym
    station_location = best_result['station_location']
    gym_location = best_result['gym_location']
    station_coords = (station_location['latitude'], station_location['longitude'])
    gym_coords = (gym_location['lat'], gym_location['lng'])

    # Get directions between locations
    directions_house_to_station = gmaps.directions(house_location, station_coords, mode='walking')
    directions_house_to_gym = gmaps.directions(house_location, gym_coords, mode='walking')
    directions_station_to_work = gmaps.directions(station_coords, work_location)

    # Add polyline routes to the map
    folium.PolyLine(get_polyline(directions_house_to_station), color="red", weight=4, opacity=0.7).add_to(map)
    folium.PolyLine(get_polyline(directions_house_to_gym), color="blue", weight=4, opacity=0.7).add_to(map)
    folium.PolyLine(get_polyline(directions_station_to_work), color="green", weight=4, opacity=0.7).add_to(map)

    # List of places to add markers for
    place_names = ["Apartment", "Station " + key_location[0], key_location[1], "Work"]
    places = [house_location, station_coords, gym_coords, work_location]

    # Add markers for each location
    for place, place_name in zip(places, place_names):
        folium.Marker([place[0], place[1]], popup=place_name).add_to(map)

    # Get midpoints of polyline routes for labels
    route_coords1 = get_polyline(directions_house_to_station)
    route_coords2 = get_polyline(directions_house_to_gym)
    route_coords3 = get_polyline(directions_station_to_work)

    mid_point1 = route_coords1[len(route_coords1) // 2]
    mid_point2 = route_coords2[len(route_coords2) // 2]
    mid_point3 = route_coords3[len(route_coords3) // 2]

    # Times at each midpoint
    times_str = [
        str(best_result['time_station']) + " min",
        str(best_result['time_gym']) + " min",
        str(best_result['time_work']) + " min"
    ]
    mid_points = [mid_point1, mid_point2, mid_point3]

    # Add time markers at midpoints with custom labels
    for mid_point, time_str in zip(mid_points, times_str):
        folium.Marker(
            location=mid_point,
            icon=DivIcon(
                icon_size=(150, 36),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 14pt; color: {line_color};">{time_str}</div>',
            )
        ).add_to(map)

    # Save the map as an HTML file
    map.save(map_name + ".html")


In [12]:
results_estation = get_stations()

In [ ]:
coor_work = coor_from_place(" ")

In [14]:
ubication_body_tech = ["Santa Maria de los angeles",
                       "vizcaya",
                       "san lucas",
                       "mall del este",
                       "city plaza centro comercial",
                       "laureles",
                       "villagrande",
                       "americas",
                       "avenida colombia",
                       "san juan",
                       "premium plaza centro comercial",
                       "niquia",
                       "robledo,"
                       "belen",
                       "camino real",
                       "llano grande"]
ubication_body_tech = ["bodytech "+gym for gym in ubication_body_tech]

In [15]:
result_maps_gym = {}

In [16]:
for gym in ubication_body_tech:
    result_maps_gym[gym] = gmaps.geocode(gym,
                                         components={"country": "CO"})

In [ ]:
ruta = ''
with open(ruta, 'w') as file:
    json.dump(result_maps_gym, file, indent=4)  

In [18]:
with open(ruta, 'r') as file:
    result_maps_gym = json.load(file)

In [19]:
result_maps_gym.keys()

dict_keys(['bodytech Santa Maria de los angeles', 'bodytech vizcaya', 'bodytech san lucas', 'bodytech mall del este', 'bodytech city plaza centro comercial', 'bodytech laureles', 'bodytech villagrande', 'bodytech americas', 'bodytech avenida colombia', 'bodytech san juan', 'bodytech premium plaza centro comercial', 'bodytech niquia', 'bodytech robledo,belen', 'bodytech camino real', 'bodytech llano grande'])

## Build Graph

In [ ]:
data_aptos = pd.read_excel('')

In [21]:
data_aptos['dirección aproximada'] = "Medellín "+data_aptos['dirección aproximada']

In [22]:
data_aptos

,SECTOR,dirección aproximada,Link
0,LA ALMERIA,Medellín Plaza campestre carrera 88a,https://www.metrocuadrado.com/inmueble/arriend...
1,BELEN,Medellín calle 29 con carrera 79a,https://www.metrocuadrado.com/inmueble/arriend...
2,SAN DIEGO,Medellín calle 42 con carrera 33,https://www.metrocuadrado.com/inmueble/arriend...
3,NaN,Medellín carrera 83 con calle 47a,https://www.metrocuadrado.com/inmueble/arriend...
4,Laureles,Medellín transversal 38 con circular 72,https://www.fincaraiz.com.co/apartamento-en-ar...
5,Conquistadores,Medellín calle 36 con carrera 66a,https://www.fincaraiz.com.co/apartamento-en-ar...
6,Urbanización quintas del sol de la mota,Medellín Urbanización quintas del sol de la mota,https://www.fincaraiz.com.co/apartamento-en-ar...
7,Belén fatima,Medellín calle 32a con carrera 65cc,https://www.fincaraiz.com.co/apartamento-en-ar...
8,Laureles,Medellín calle 43 # 77-79,link_vacio


In [23]:
data_aptos[['lat', 'lng']] = data_aptos['dirección aproximada'].apply(coor_from_place).apply(pd.Series)

In [53]:
data_aptos[["SECTOR","dirección aproximada","lat","lng"]]

,SECTOR,dirección aproximada,lat,lng
0,LA ALMERIA,Medellín Plaza campestre carrera 88a,6.242865,-75.615143
1,BELEN,Medellín calle 29 con carrera 79a,6.230520,-75.599630
2,SAN DIEGO,Medellín calle 42 con carrera 33,6.237209,-75.557392
3,NaN,Medellín carrera 83 con calle 47a,6.257647,-75.600786
4,Laureles,Medellín transversal 38 con circular 72,6.240972,-75.593893
5,Conquistadores,Medellín calle 36 con carrera 66a,6.242306,-75.585247
6,Urbanización quintas del sol de la mota,Medellín Urbanización quintas del sol de la mota,6.212546,-75.598519
7,Belén fatima,Medellín calle 32a con carrera 65cc,6.235597,-75.585028
8,Laureles,Medellín calle 43 # 77-79,6.249235,-75.596714


In [49]:
data_aptos["SECTOR"][8]

'Laureles'

In [27]:
for dict_result in tqdm(results_estation):
    for i in range(len(data_aptos)):
        origen = (data_aptos['lat'][i],data_aptos['lng'][i])
        destino = (dict_result['latitude'],dict_result['longitude'])
        dict_result[i] = get_time_from_coor(origen,destino,mode='walking')

100%|██████████| 60/60 [01:17<00:00,  1.29s/it]


In [31]:
dict_data_aptos_result = {'stations':{},'gym':{},'work':{}}
for i in tqdm(range(len(data_aptos))):
    origen = (data_aptos['lat'][i], data_aptos['lng'][i])

    dict_distance_estations_i = {}
    for dict_result in results_estation:
        point_station = (dict_result['latitude'], dict_result['longitude'])
        time_to_station = get_time_from_coor(origen, point_station, mode='walking')
        dict_distance_estations_i[dict_result['name']] = time_to_station  

    dict_data_aptos_result['stations'][i] = dict_distance_estations_i


    dict_distance_gym_i = {}
    for gym in result_maps_gym.keys():
        result_ubication = result_maps_gym[gym][0]['geometry']['location']
        point_gym = (result_ubication['lat'], result_ubication['lng'])
        time_to_gym = get_time_from_coor(origen, point_gym, mode='walking')
        dict_distance_gym_i[gym] = time_to_gym      

    dict_data_aptos_result['gym'][i] = dict_distance_gym_i

dict_distance_station_work = {}
for dict_result in results_estation:
    point_station = (dict_result['latitude'], dict_result['longitude'])
    time_to_work = get_time_from_coor(point_station, coor_work)
    dict_distance_station_work[dict_result["name"]] = time_to_work    

dict_data_aptos_result['work'] = dict_distance_station_work

100%|██████████| 9/9 [01:31<00:00, 10.11s/it]


In [32]:
with open('dict_data_aptos_result.json', 'w', encoding='utf-8') as f:
    json.dump(dict_data_aptos_result, f, ensure_ascii=False, indent=4)

In [33]:
with open('dict_data_aptos_result.json', 'r', encoding='utf-8') as f:
    dict_data_aptos_result = json.load(f)

In [34]:
len(dict_data_aptos_result['stations']['1'].keys())

59

In [35]:
dict_data_aptos_min = dict_data_aptos_result.copy()

In [36]:
for i in tqdm(range(9)):
    dict_gym = dict_data_aptos_min['gym'][str(i)]
    for gym in dict_gym.keys():
        dict_gym[gym]=str_to_minutes(dict_gym[gym])
    dict_stations = dict_data_aptos_min['stations'][str(i)]
    for station in dict_stations.keys():
        dict_stations[station]=str_to_minutes(dict_stations[station])
for station in dict_data_aptos_min['work'].keys():
    dict_data_aptos_min['work'][station]=str_to_minutes(dict_data_aptos_min['work'][station])

100%|██████████| 9/9 [00:00<00:00, 3279.93it/s]


In [80]:
dict_result_cost ={str(i):get_cost_place(dict_data_aptos_min,
                                         result_maps_gym,
                                         results_estation,
                                         i,weight_station=1,weight_gym=5) for i in range(9)}

In [82]:
dict_result_cost_sorted = get_top_n_keys(dict_result_cost,n=1)

In [84]:
dict_best_result = sorted_time_cost(dict_result_cost_sorted)

In [86]:

rows = []

# Iterar sobre el diccionario
for key, value in dict_best_result.items():
    for location, info in value.items():
        row = {
            'Ubicación': key,
            'Sector':data_aptos["SECTOR"][int(key)],
            'station': location[0],
            'gym': location[1],
            'time_cost': info['time_cost'],
            'time_station': info['time_station'],
            'time_gym': info['time_gym'],
            'time_work': info['time_work']
        }
        rows.append(row)

# Crear el DataFrame
df = pd.DataFrame(rows)
df

,Ubicación,Sector,station,gym,time_cost,time_station,time_gym,time_work
0,3,NaN,Santa Lucía Station,bodytech san juan,122,6,13,51
1,6,Urbanización quintas del sol de la mota,Estación poblado del metro,bodytech americas,134,45,11,34
2,8,Laureles,Estadio,bodytech san juan,141,17,15,49
3,7,Belén fatima,Estación Transferencia Metro Plus Industriales,bodytech premium plaza centro comercial,193,20,27,38
4,5,Conquistadores,Exposiciones Cra.51 #36-61 Medellin,bodytech laureles,195,23,27,37
5,0,LA ALMERIA,Santa Lucía Station,bodytech san juan,198,32,23,51
6,1,BELEN,Rosales,bodytech americas,208,13,29,50
7,2,SAN DIEGO,Estación Metro San Antonio,bodytech camino real,210,26,29,39
8,4,Laureles,Exposiciones Cra.51 #36-61 Medellin,bodytech san juan,221,34,30,37


In [87]:
df.to_csv("results.csv")

## Gráfico

In [95]:
i=6
generate_map(dict_best_result,i,(data_aptos.iloc[i]['lat'],data_aptos.iloc[i]['lng']),coor_work,"result_"+str(i),line_color='black')